In [1]:
import pandas as pd
import numpy as np
from numpy.linalg import norm
from numpy import dot
import fasttext
import ast
import string
import spacy
import nltk
from nltk.corpus import stopwords
import copy
import re
from collections import Counter
from nltk.stem import WordNetLemmatizer
import symspellpy
from nltk.stem import PorterStemmer

### Eperiemented few them haven't used all
    - just attached them incase if you wanna play with

In [2]:
#spell correction package tries to retrieve the nearest words which would help reducing the noise
spell_correction = symspellpy.SymSpell(max_dictionary_edit_distance=2,count_threshold=4)

#Engineer, Engineering, and etc would be back to their root words for matching
lemmatizer = WordNetLemmatizer()

#removing stop words from degree and specialization
stop_words = set(stopwords.words('english'))

#experiementing with spacy lemmas as well
nlp = spacy.load('en_core_web_md')

#stemming
stem = PorterStemmer()

In [3]:
def get_lemma(text):
    """
    getting lemma from spacy model
    """
    text = text.strip()
    if len(text)==0:
        return []
    return [token.lemma_ for token in nlp(text)]

In [4]:
def get_porter_lemma(text):
    """
    getting lemma or root from nltk model
    """
    return lemmatizer.lemmatize("systems")

In [5]:
def get_similarity_nlp(sent_a,sent_b):
    """
    getting similarity between docs form spacy model
    """
    a = nlp(sent_a)
    b = nlp(sent_b)
    return a.similarity(b)

In [6]:
replace_data = ["master of science in", "bachelor of science in", "master of science", "bachelor of science", 
                "bachelors in", "bachelor's in", "bachelors", "masters in", "master's in", "masters", 
                "bs in", "ms in", "ms", "m.s in", "m.s", "b.s in", "b.s", "bs in", "bs", "bachelor of arts", "gpa", 
                "bba", "b.b.a", "mba", "m.b.a", "b.a", "ba","major in", "major", "minor in", "minor",
                "concentration in", "concentration", "master's", "masters", "bachelor's", "bachelors", 
                "n/a", "master", "bachelor", "degree", "certificate"]

        

#### Cleaning degree data wrt mentor so it can be matched to help topics
    - etl operations removing stop words from degree data
    - tried get_similarity_nlp as well as it taking more time just worked on exact text match nothing fancy

In [7]:
def clean_data(val):
    val = val.strip()
    out = []
    if val!="":
        if val.endswith('.'):
            val = val[:-1]
        if val == "n/a":
            val = ""
        temp = val.split()
        while val and val[0] not in string.ascii_letters:
            val = val[1:]
        while val and val[-1] not in string.ascii_letters:
            val = val[:-1]
        val = " ".join(temp)
        val = val.lower()
        val = val.strip()
        val = val.replace(" w/ "," and ")
        val = val.replace("&","and")
        val = val.replace("/"," ")
        val = val.replace("-"," ")
        val = re.sub('[^A-Z^a-z^0-9^\s]+','',val)
        temp = val.split()
        temp = [e.strip() for e in temp]
        val = " ".join(temp)
        for d in replace_data:
            if val.startswith(d+" ") or " "+d+" " in val or val.endswith(" "+d):
                val = val.replace(d+" ","")
                val = val.strip()
                while val and val[0] not in string.ascii_letters:
                    val = val[1:]
                temp = val.split()
                temp = [e.strip() for e in temp]
                if len(temp)>1:
                    while temp and temp[0] in stop_words:
                        temp.pop(0)
                    while temp and temp[-1] in stop_words:
                        temp.pop()
                    val = " ".join(temp)
            val = " ".join(temp)
        while val and val[0] not in string.ascii_letters:
            val = val[1:]
        while val and val[-1] not in string.ascii_letters:
            val = val[:-1]
        val = val.strip()
        val = val.split()
        for v in val:
            if v not in stop_words and len(v)>2:
                out.append(v)
        return out
    return out

In [8]:
df = pd.read_csv('pg_recommendation_data.csv')
df.head(n=5)

,mentee_major,mentee_help_topics,mentee_experitse,mentor_major,mentor_help_topics,mentor_experitse,final_match
0,['Accounting and Financial Management'],"['Life Skills', 'Cover Letter Review', 'Career...",['Accounting'],['Accounting and Financial Management'],"['Parenting vs. Career', 'Career / Industry Tr...","['Finance', 'Accounting']",1
1,['Information Systems Management'],"['Working Overseas', 'Life Skills', 'Startups'...","['Food & Restaurants', 'Law', 'Computer - IT S...","['Information Systems Management', 'Project Ma...","['Leadership Skills', 'Working Overseas', 'Per...","['Trade (Wholesale)', 'Government / Public Adm...",0
2,"['Cybersecurity Technology', 'Computer Network...","['Leadership Skills', 'Personal and Profession...","['Military & Defense', 'Technology', 'Computer...","['Cybersecurity Technology', 'Computer Network...","['Management', 'Resume / CV Review', 'Career /...","['Government / Public Admin', 'Computer - IT S...",1
3,['Masters of Human Resources Management'],[],['Human Resources'],['Business Administration'],"['Leadership Skills', 'Life Skills', 'Intervie...","['Human Resources', 'Advertising / Marketing',...",0
4,['Sociology'],"['Leadership Skills', 'Life Skills', 'Manageme...",['Healthcare'],['Business Administration'],"['Leadership Skills', 'Working Overseas', 'Per...","['Human Resources', 'Computer - Hardware', 'Fi...",0


In [9]:
cols = df.columns
cols

Index(['mentee_major', 'mentee_help_topics', 'mentee_experitse',
       'mentor_major', 'mentor_help_topics', 'mentor_experitse',
       'final_match'],
      dtype='object')

In [10]:
for col in cols[:-1]:
    df[col] = df[col].apply(lambda x: ast.literal_eval(x))

In [11]:
import itertools

In [12]:
for col in cols[:-1]:
    df[col] = df[col].apply(lambda l:list(itertools.chain.from_iterable([clean_data(x) for x in l])))

In [13]:
df_cleaned = df[(df["mentee_help_topics"].str.len()>0) & 
   ((df["mentor_help_topics"].str.len()>0) | 
    (df["mentor_experitse"].str.len()>0) | 
    (df["mentor_major"].str.len()>0))]

In [14]:
df_cleaned.shape

(32752, 7)

In [15]:
docs,features = df.shape

In [16]:
s = symspellpy.SymSpell(max_dictionary_edit_distance=2,count_threshold=4) #.create_dictionary_entry()

In [17]:
def get_nearest_words(s,word,verbose=10,dist=2):
    out = []
    o = s.lookup(word,verbosity=verbose,max_edit_distance=dist)
    if len(o)>0:
        out = [(t.term,t.count,t.distance) for t in o]
    return out

In [18]:
def get_nearest_words(s,word,verbose=10,dist=2):
    out = []
    o = s.lookup(word,verbosity=verbose,max_edit_distance=dist)
    if len(o)>0:
        out = [(t.term,t.count,t.distance) for t in o]
    return out

#### Helped with fixing typo error from users like systerm and many more

In [19]:
def create_spellcheck(s,d):
    for key,val in sorted(d.items(),key=lambda x:-x[1]):
        if val>10:
            s.create_dictionary_entry(key,val)
        else:
            output = get_nearest_words(s,key)
            if len(output)==0:
                s.create_dictionary_entry(key,val)
            else:
                max_count = -1
                update_term = None
                min_dist = float("inf")
                for term,count,dist in output:
                    if dist<=min_dist:
                        min_dist = dist
                        if count>max_count:
                            max_count = count
                            update_term = term
                max_count += val
                s.create_dictionary_entry(update_term,max_count)
    return s

In [20]:
def get_nearest_words_1(s,word,verbose=10,dist=2):
    o = s.lookup(word,verbosity=verbose,max_edit_distance=dist)
    min_occ = -float("inf")
    out = word
    distance = float("inf")
    for t in o:
        if t.count>10 and min_occ<t.count:
            min_occ = t.count
            out = t.term
    return out

In [21]:
s = symspellpy.SymSpell(max_dictionary_edit_distance=2,count_threshold=3) #.create_dictionary_entry()
for col in cols[:-1]:
    d = Counter(itertools.chain.from_iterable(list(df_cleaned[cols[0]])))
    s = create_spellcheck(s,d)
    #print("done")
    df_cleaned[col] = df_cleaned[col].apply(lambda x:[get_nearest_words_1(s,w) for w in x])

<ipython-input-21-ddb1ab505372>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned[col] = df_cleaned[col].apply(lambda x:[get_nearest_words_1(s,w) for w in x])


In [187]:
document_frequency = dict()
term_frequency = dict()

In [188]:
for col in cols[:-1]:
    document_frequency[col] = dict()
    term_frequency[col] = dict()

In [189]:
for col in cols[:-1]:
    data = df[col]
    i  = 0
    for i,vals in enumerate(data):
        for v in vals:
            if v in document_frequency[col]:
                document_frequency[col][v].add(i)
            else:
                document_frequency[col][v] = set()
                document_frequency[col][v].add(i)
            if v in term_frequency[col]:
                term_frequency[col][v] += 1
            else:
                term_frequency[col][v] = 1

##### used both Tf-idf to give importance wrt word and exact match couldn't find much differnce updated results at the bottom

In [22]:
def match_logic(mentee_hp,mentor_hp):
    mentee_dict = Counter(mentee_hp)
    mentor_dict = Counter(mentor_hp)
    if len(mentee_hp)==0 or len(mentor_hp)==0:
        return 0
    score = 0
    for key,val in mentee_dict.items():
        if key in mentor_dict:
            tf = 1
            try:
                tf = val/term_frequency["mentee_help_topics"].get(key)
            except Exception as e:
                print(e)
                print(key)
            idf = np.log(docs)
            try:
                idf = np.log(docs/len(document_frequency["mentee_help_topics"].get(key)))
            except Exception as e:
                print(e)
                print(key)
            score = score + (tf*idf)
    score = score*1000
    return score

In [23]:
def match_vals(a,b):
    match = 0
    if len(a)==0 or len(b)==0:
        return 0
    total = len(b)
    for word_a in a:
        for word_b in b:
            if word_a==word_b:
                match += 1
    return match

In [24]:
cols

Index(['mentee_major', 'mentee_help_topics', 'mentee_experitse',
       'mentor_major', 'mentor_help_topics', 'mentor_experitse',
       'final_match'],
      dtype='object')

In [25]:
get_similarity_nlp("data science","machine learning")

0.49251779011189367

#### Problems with Spacy and pre-trained vectors:
    - They weren't able to help me with similary words are bad, not sure how to choose threshold.
    - We need to re-train word2vec embedding wrt our domain might be jobs or degree with description using Skip gram or if we have dictonary we can use it as well would increase confidence a lot which be re-used for future purposes
    - you can see the above example as it can't say data science and Machie learnig similiary score should be same, same results were wrt glove and fastext as well slightly better still not that great

#### Matching logic:
    - Mentee help topic and Mentee expertise are the are he is trying to improve.
    - I tried matching the following:
        - Mentee help topics with Mentor
        - Mentee help topics with Mentor expertise and degree
        - Mentee expertise which he is looking gain with Mentor help topic, expertise and degree. 
        - I gave more importance to help topics and expertise but there wasn't much improvement in results
        - I didn't find importance wrt matching Mentee degree wrt other

In [26]:
df["match_help_topics"] = np.vectorize(match_vals)(
    df["mentee_help_topics"],
    df["mentor_help_topics"])

In [27]:
df["match_ht_expertise"] = np.vectorize(match_vals)(
    df["mentee_help_topics"],
    df["mentor_experitse"])

In [28]:
df["match_ht_major"] = np.vectorize(match_vals)(
    df["mentee_help_topics"],
    df["mentor_major"])

In [29]:
df["match_expertise_hp"] = np.vectorize(match_vals)(
    df["mentee_experitse"],
    df["mentor_help_topics"])

In [30]:
df["match_expertise_expertise"] = np.vectorize(match_vals)(
    df["mentee_experitse"],
    df["mentor_experitse"])

In [31]:
df["match_expertise_major"] = np.vectorize(match_vals)(
    df["mentee_experitse"],
    df["mentor_major"])

In [32]:
df.columns

Index(['mentee_major', 'mentee_help_topics', 'mentee_experitse',
       'mentor_major', 'mentor_help_topics', 'mentor_experitse', 'final_match',
       'match_help_topics', 'match_ht_expertise', 'match_ht_major',
       'match_expertise_hp', 'match_expertise_expertise',
       'match_expertise_major'],
      dtype='object')

###  Tried giving more weight to help topics and expertise match but there isn't much difference
    df["match_help_topics"] = df["match_help_topics"]*3
    df["match_expertise_ht"] = df["match_expertise_ht"]*2
    df["match_expertise_expertise"] = df["match_expertise_expertise"]*3
    df["match_expertise_hp"] = df["match_expertise_hp"]*2
    df["match_expertise_ht"] = df["match_expertise_ht"]*2
        - commented it so you can verify if you feel like

In [82]:
df

,mentee_major,mentee_help_topics,mentee_experitse,mentor_major,mentor_help_topics,mentor_experitse,final_match,match_help_topics,match_expertise_ht,match_major_ht
0,"[accounting, financial, management]","[life, skills, cyber, letters, review, career,...",[accounting],"[accounting, financial, management]","[parenting, career, career, industry, trends]","[finance, accounting]",1,0.840836,0,0
1,"[information, systems, management]","[working, overseas, life, skills, startups, ca...","[food, restaurant, law, computer, services, co...","[information, systems, management, project, ma...","[leadership, skills, working, overseas, person...","[trade, wholesale, government, public, admin, ...",0,14.898326,0,0
2,"[cybersecurity, technology, computer, networks...","[leadership, skills, personnel, professional, ...","[military, defense, technology, computer, serv...","[cybersecurity, technology, computer, networks...","[management, resouce, revised, career, industr...","[government, public, admin, computer, services...",1,1.927130,0,0
3,"[human, resources, management]",[],"[human, resources]","[business, administration]","[leadership, skills, life, skills, interviewin...","[human, resources, advertising, marketing, tel...",0,0.000000,0,0
4,[sociology],"[leadership, skills, life, skills, management,...",[healthcare],"[business, administration]","[leadership, skills, working, overseas, person...","[human, resources, computer, hardware, finance...",0,22.209784,0,0
...,...,...,...,...,...,...,...,...,...,...
48877,[communications],[],"[government, public, admin, business, services...","[business, administration]","[leadership, skills, life, skills, management,...","[sports, recreation, education, entertainment,...",0,0.000000,0,0
48878,"[business, administration]","[interviewing, tips, web, earth, strategic, re...","[business, services, consulting, transportatio...",[finance],[],"[international, affairs, development]",1,0.000000,0,0
48879,"[high, school, management, studies, general, s...","[leadership, skills, life, skills, management,...","[non, profit, philanthropy, government, public...","[management, studies]",[],[],1,0.000000,0,0
48880,"[environmental, management]","[general, networking, career, exchange, advice...","[environmental, entry]","[environmental, management]","[leadership, skills, working, overseas, life, ...","[entertainment, entry, international, affairs,...",0,0.840836,0,0


In [36]:
test = df[(df["mentee_help_topics"].str.len()>0 | (df["mentee_experitse"].str.len()>0)) & 
   (((df["mentor_help_topics"].str.len()>0) ) | (df["mentor_experitse"].str.len()>0)
   | (df["mentor_major"].str.len()>0))
         ][['match_help_topics', 'match_ht_expertise', 'match_ht_major',
       'match_expertise_hp', 'match_expertise_expertise',
       'match_expertise_major',"final_match"]]

In [226]:
test = df

In [37]:
#X = test[["match_help_topics","match_expertise_ht","match_major_ht"]]
X = test[['match_help_topics', 'match_ht_expertise', 'match_ht_major',
       'match_expertise_hp', 'match_expertise_expertise',
       'match_expertise_major']]
Y = test[['final_match']]

In [38]:
test.shape

(32382, 7)

In [41]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [42]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.20,random_state=0)


In [43]:
logreg = LogisticRegression()
# fit the model with data
logreg.fit(X_train,Y_train)



/Applications/anaconda3/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


LogisticRegression()

In [44]:
y_pred=logreg.predict(X_test)



In [45]:
cnf_matrix = metrics.confusion_matrix(Y_test, y_pred)
cnf_matrix

array([[  56, 2272],
       [  50, 4099]])

In [46]:
(cnf_matrix[0][0]+cnf_matrix[1][1])/(sum(cnf_matrix[0])+sum(cnf_matrix[1]))

0.6415006947660954

##### using Tf-Idf accuracy:
    0.6363913906273851
#### Accuracy with just match:
    - without removing noise records
        0.5826940779380178
    - removing nosie records
        0.6415006947660954
        

##### Tried stemming and tokenization as well, it wasn't that great, please feel free to make slight modifications wrt code.
    - don't try spacy for lemmatization it's time consuming and might exhaust you, explore dusk.

In [47]:
import pickle
filename = 'lg_regression.sav'
pickle.dump(logreg, open(filename, 'wb'))

#### Tried exact using stemming and lemmatization:
    - Results were bad and there isn't any information. There was only difference of less than 1-2% Financial lemma wrt nltk is financi, which really tough to match to any word. Found few more issues like that, so haven't gone with that approach 

##### Sorry for the delay. Due to multiple interviews, and few personal things to attend I couldn't finish is asap.